In [36]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
import datetime
import scipy as sc
from scipy import stats
import matplotlib.pyplot

from flask import render_template, request, jsonify
import flask
import traceback
import pickle
from flask import Flask
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, STATUS_FAIL

import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, KFold

from sklearn.utils.multiclass import type_of_target

from sklearn import preprocessing

from scipy.stats import describe as des

import matplotlib.pylab as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
#import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import io
import base64


import plotly
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

from flask import render_template
from bokeh.plotting import figure
from bokeh.embed import components

import json

In [3]:
df=pd.read_csv('E:\\datasets\\baum\\pool_free_space 22.09.20-04.12.20.csv', ';')
df.head()

,itemid,clock,value,name
0,6087991,1604997154,"3952395092788,00",Pool free space: Pool49
1,6087991,1604997184,"3952394989876,00",Pool free space: Pool49
2,6087991,1604997214,"3952394754868,00",Pool free space: Pool49
3,6087991,1604997244,"3952395080500,00",Pool free space: Pool49
4,6087991,1604997274,"3952394868532,00",Pool free space: Pool49


Сделаем 'feature engineering' для времени и даты

In [4]:
df['datetime']=df['clock']\
.apply(lambda x: (datetime.datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S'))
df['date']=df['clock']\
.apply(lambda x: (datetime.datetime.fromtimestamp(x)).strftime('%Y-%m-%d'))
df.head()

,itemid,clock,value,name,datetime,date
0,6087991,1604997154,"3952395092788,00",Pool free space: Pool49,2020-11-10 11:32:34,2020-11-10
1,6087991,1604997184,"3952394989876,00",Pool free space: Pool49,2020-11-10 11:33:04,2020-11-10
2,6087991,1604997214,"3952394754868,00",Pool free space: Pool49,2020-11-10 11:33:34,2020-11-10
3,6087991,1604997244,"3952395080500,00",Pool free space: Pool49,2020-11-10 11:34:04,2020-11-10
4,6087991,1604997274,"3952394868532,00",Pool free space: Pool49,2020-11-10 11:34:34,2020-11-10


In [5]:
#df['value']
df['datetime']=pd.to_datetime(df['datetime'])
df['date']=pd.to_datetime(df['date'])

In [7]:
df['hour']=df['datetime'].dt.hour
df['dayofweek']=df['datetime'].dt.dayofweek
df['quarter']=df['datetime'].dt.quarter
df['month']=df['datetime'].dt.month
df['year']=df['datetime'].dt.year
df['dayofyear']=df['datetime'].dt.dayofyear
df['dayofmonth']=df['datetime'].dt.day
df['weekofyear']=df['datetime'].dt.weekofyear
df['minute']=df['datetime'].dt.minute
df['second']=df['datetime'].dt.second

e:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


Данные плохо качества (у числового поля формат - object), надо трансформировать в число.

In [6]:
df['value']=df['value'].apply(lambda x: float(x.replace(',','.')))

Добавим категориальных переменных

In [8]:
df=pd.get_dummies(df, columns=['itemid', 'name'])

In [9]:
df.head()

,clock,value,datetime,date,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minute,second,itemid_6086590,itemid_6086591,itemid_6087991,itemid_6088159,name_Pool free space: Pool49,name_Pool free space: Pool50
0,1604997154,"3952395092788,00",2020-11-10 11:32:34,2020-11-10,11,1,4,11,2020,315,10,46,32,34,0,0,1,0,1,0
1,1604997184,"3952394989876,00",2020-11-10 11:33:04,2020-11-10,11,1,4,11,2020,315,10,46,33,4,0,0,1,0,1,0
2,1604997214,"3952394754868,00",2020-11-10 11:33:34,2020-11-10,11,1,4,11,2020,315,10,46,33,34,0,0,1,0,1,0
3,1604997244,"3952395080500,00",2020-11-10 11:34:04,2020-11-10,11,1,4,11,2020,315,10,46,34,4,0,0,1,0,1,0
4,1604997274,"3952394868532,00",2020-11-10 11:34:34,2020-11-10,11,1,4,11,2020,315,10,46,34,34,0,0,1,0,1,0


In [10]:
xtrain, xtest, ytrain, ytest=train_test_split(df[['itemid_6086590', 'itemid_6086591', 'itemid_6087991',
       'itemid_6088159', 'name_Pool free space: Pool49',
       'name_Pool free space: Pool50', 'hour',
       'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth',
       'weekofyear', 'minute', 'second']], df['value'], test_size=0.15)


Не понравились мне результаты грид серча, решил вот попробовать hyperopt и вроде по метрике MSE дала на несколько порядков лучше показатели, да ещё и за короткое время относительно.

In [11]:
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.41, 0.05)),
    'gamma':            hp.choice('gamma',            np.arange(0.05, 0.41, 0.05)),
    'reg_alpha':        hp.choice('reg_alpha',        np.arange(0.05, 1, 0.05)),
    'reg_lambda':       hp.choice('reg_lambda',       np.arange(0.05, 1, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(2, 20, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 12, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     hp.choice('n_estimators',     np.arange(10, 500, 10)),
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


In [12]:
class HPOpt(object):

    def __init__(self, xtrain, xtest, ytrain, ytest):
        self.x_train = xtrain
        self.x_test  = xtest
        self.y_train = ytrain
        self.y_test  = ytest

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [ ]:
obj = HPOpt(xtrain, xtest, ytrain, ytest)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

In [204]:
xgb_opt

({'colsample_bytree': 4,
  'learning_rate': 4,
  'max_depth': 9,
  'min_child_weight': 1,
  'subsample': 0.9634691113577399},
 <hyperopt.base.Trials at 0x22f615d67c8>)

In [206]:
obj = HPOpt(xtrain, xtest, ytrain, ytest)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

100%|███████████████████████████████████████████████| 100/100 [34:33<00:00, 20.74s/trial, best loss: 7180562539.446937]


In [207]:
xgb_opt

({'colsample_bytree': 3,
  'gamma': 1,
  'learning_rate': 4,
  'max_depth': 10,
  'min_child_weight': 2,
  'n_estimators': 41,
  'reg_alpha': 5,
  'reg_lambda': 13,
  'subsample': 0.9616547645808519},
 <hyperopt.base.Trials at 0x22f6d935bc8>)

In [28]:
print (np.arange(0.3, 0.8, 0.1)[3], np.arange(0.05, 0.41, 0.05)[1], np.arange(0.05, 0.41, 0.05)[4],
np.arange(2, 20, 1, dtype=int)[10], np.arange(1, 12, 1, dtype=int)[2], np.arange(10, 500, 10)[41],
np.arange(0.05, 1, 0.05)[5], np.arange(0.05, 1, 0.05)[13], 0.9616547645808519)

0.6000000000000001 0.1 0.25 12 3 420 0.3 0.7000000000000001 0.9616547645808519


Обучаем модель на получившихся параметрах.

In [29]:
xgbr = xgb.XGBRegressor(colsample_bytree=0.6, gamma=0.1, learning_rate=0.25, max_depth=12, 
                        min_child_weight=3, n_estimators=420, reg_alpha=0.3, reg_lambda=0.7, subsample=0.9616547645808519) 
xgbr.fit(xtrain, ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.25, max_delta_step=0, max_depth=12,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=420, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0.3, reg_lambda=0.7, scale_pos_weight=1,
             subsample=0.9616547645808519, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [221]:
xgb_opt

({'colsample_bytree': 3,
  'gamma': 1,
  'learning_rate': 4,
  'max_depth': 10,
  'min_child_weight': 2,
  'n_estimators': 41,
  'reg_alpha': 5,
  'reg_lambda': 13,
  'subsample': 0.9616547645808519},
 <hyperopt.base.Trials at 0x22f6d935bc8>)

Подготовим данные для тестирования модели в приложении

In [70]:
df1=pd.read_csv('E:\\datasets\\baum\\pool_free_space 22.09.20-04.12.20.csv', ';')
df1.loc[xtest.index].drop(columns='value').to_json('E:\\datasets\\baum\\test.json', orient='records')

In [71]:
test=pd.read_json('E:\\datasets\\baum\\test.json')#, orient='records')

In [50]:
pickle.dump(xgbr, open('E:\\datasets\\baum\\xgbr.txt', "wb"))

In [51]:
xgb_model_loaded = pickle.load(open('E:\\datasets\\baum\\xgbr.txt', "rb"))


In [23]:
final_predictions=pd.read_csv('E:\\datasets\\baum\\pool_free_space 22.09.20-04.12.20.csv', ';', nrows=0)
final_predictions

,itemid,clock,value,name


Приложение для отрисовки данных полученных данных

In [42]:
app = Flask(__name__, template_folder='E:\\datasets\\baum\\',
            static_folder='E:\\datasets\\baum\\', static_url_path = "")#__name__

@app.route('/dashboard', methods=['GET'])
#def main():
#return ('time series from Baum Prediction')
def chartTest():
    global final_predictions
    
    df=pd.read_csv('E:\\datasets\\baum\\pool_free_space 22.09.20-04.12.20.csv', ';')
    df['date']=df['clock'].apply(lambda x: (datetime.datetime.fromtimestamp(x)).strftime('%Y-%m-%d'))
    df['date']=pd.to_datetime(df['date'])
    df['value']=df['value'].apply(lambda x: float(x.replace(',','.')))
    #df=df.sort_values(by='date')
    df=df.groupby(['date'])['value'].sum().reset_index()
    data=final_predictions.copy()
    data['date']=data['clock']\
    .apply(lambda x: (datetime.datetime.fromtimestamp(x)).strftime('%Y-%m-%d'))
    data['date']=pd.to_datetime(data['date'])
    data=data.groupby(['date'])['value'].sum().reset_index()
    
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    axis.set_title("title")
    axis.set_xlabel("x-axis")
    axis.set_ylabel("y-axis")
    axis.grid()
    axis.plot(data['date'], data['value'], color = 'green')
    axis.plot(df['date'], df['value'], color = 'red')
    #lnprice=np.log(price)
    #plt.plot(df['date'], df['value'], color = 'red')  
    
    #plt.plot(data['date'], data['value'], color = 'green')   
    #plt.savefig('new_plot.png')
    pngImage = io.BytesIO()
    FigureCanvas(fig).print_png(pngImage)
    
    # Encode PNG image to base64 string
    pngImageB64String = "data:image/png;base64,"
    pngImageB64String += base64.b64encode(pngImage.getvalue()).decode('utf8')
    

    return render_template('untitled1.html', name = 'График временного ряда', image=pngImageB64String)#, url ='new_plot.png')   


@app.route('/predict', methods=['POST'])
#def main():
   # return ('time series from Baum Prediction')

def apicall():
    """API Call

    Pandas dataframe (sent as a payload) from API Call
    """
    global final_predictions 
    try:
        test_json = request.get_json()
        print (test_json)
        test1 = pd.read_json(test_json, orient='records')
        test=test1.copy()

        #To resolve the issue of TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'
        test['datetime']=test['clock']\
        .apply(lambda x: (datetime.datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S'))
        test['date']=test['clock']\
        .apply(lambda x: (datetime.datetime.fromtimestamp(x)).strftime('%Y-%m-%d'))
        
        
        test['datetime']=pd.to_datetime(test['datetime'])
        test['date']=pd.to_datetime(test['date'])

        test['datetime']=pd.to_datetime(test['datetime'])
        test['date']=pd.to_datetime(test['date'])
        test['hour']=test['datetime'].dt.hour
        test['dayofweek']=test['datetime'].dt.dayofweek
        test['quarter']=test['datetime'].dt.quarter
        test['month']=test['datetime'].dt.month
        test['year']=test['datetime'].dt.year
        test['dayofyear']=test['datetime'].dt.dayofyear
        test['dayofmonth']=test['datetime'].dt.day
        test['weekofyear']=test['datetime'].dt.weekofyear
        test['minute']=test['datetime'].dt.minute
        test['second']=test['datetime'].dt.second
        
        test=pd.get_dummies(test, columns=['itemid', 'name'])
        #Getting the Loan_IDs separated out
        #loan_ids = test['Loan_ID']

    except Exception as e:
        raise e

    #reg = 'model_v1.pk'

    if test.empty:
        return(bad_request())
    else:
        #Load the saved model
        print("Loading the model...")
        #loaded_model = None
        loaded_model = pickle.load(open('E:\\datasets\\baum\\xgbr.txt', "rb"))
        

        print("The model has been loaded...doing predictions now...")
        #print (test.dtypes)
        #print (test['clock'].iloc[:5])
        l=['itemid_6086590', 'itemid_6086591', 'itemid_6087991', 'itemid_6088159', 
           'name_Pool free space: Pool49',
           'name_Pool free space: Pool50', 'hour', 'dayofweek', 'quarter', 'month',
           'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'minute', 'second']#list(test.columns)[3:]
        for i in l:
            if i not in test:
                test[i]=0
        predictions = loaded_model.predict(test[l])

        """Add the predictions as Series to a new pandas dataframe
                                OR
           Depending on the use-case, the entire test data appended with the new files
        """
        test1['value']=predictions
        final_predictions=test1
        
        """We can be as creative in sending the responses.
           But we need to send the response codes as well.
        """
        responses = jsonify(final_predictions.to_json(orient="records"))
        
        responses.status_code = 200

        return (responses)
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Mar/2021 11:35:00] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2021 11:35:07] "GET /dashboard HTTP/1.1" 200 -
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

e:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


Loading the model...
The model has been loaded...doing predictions now...


127.0.0.1 - - [04/Mar/2021 11:35:26] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2021 11:35:31] "GET /dashboard HTTP/1.1" 200 -
[2021-03-04 11:42:32,818] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "e:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "e:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "e:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "e:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "e:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "e:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    retu

E:\datasets\baum\test.json


...

127.0.0.1 - - [04/Mar/2021 11:42:56] "POST /predict HTTP/1.1" 200 -
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

e:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


Loading the model...
The model has been loaded...doing predictions now...


127.0.0.1 - - [04/Mar/2021 11:43:32] "POST /predict HTTP/1.1" 200 -
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

e:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


Loading the model...
The model has been loaded...doing predictions now...


127.0.0.1 - - [04/Mar/2021 11:43:40] "POST /predict HTTP/1.1" 200 -
